In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

# 컨텍스트 밴딧

여기에서 우리는 컨텍스트 밴딧을 정의합니다. 

이 예제에서는 각각 4개의 손잡이를 가지는 3개의 밴딧을 이용하도록 하겠습니다. 각각의 밴딧은 각각의 손잡이에 대해 다른 성공 확률을 가지기 때문에 최고의 결과를 얻어내기 위해서는 다른 액션이 요구됩니다. pullBandit 함수는 평균 0의 정규분포로부터 랜덤한 수를 발생시킵니다. 밴딧의 수가 낮을수록 양의 보상이 반환될 가능성이 큽니다. 우리는 에이전트가 주어진 밴딧에 대해 가장 자주 양의 보상을 주는 손잡이를 항상 선택하는 방법을 학습하기를 원하는 것입니다. 

In [2]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        # 밴딧의 손잡이 목록을 만들어 준다. 현재 손잡이 4, 2, 그리고 1이 각각 최적이다.
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits)) #각각의 에피소드에 대해 랜덤한 상태를 반환
        return self.state
        
    def pullArm(self,action):
        #랜덤한 수를 얻는다
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            #양의 보상을 반환한다
            return 1
        else:
            #음의 보상을 반환한다
            return -1

# 정책 기반 에이전트

아래 코드를 통해 간단한 신경 에이전트를 만들어 보겠습니다. 

입력으로 현재의 상태를 받아 액션을 반환합니다. 이렇게 함으로써 에이전트는 환경의 상태를 조건으로 액션을 취하게 되는데, 이는 완전한 강화학습 문제를 해결하기 위해 한걸음 더 나아가는 중요한 단계입니다. 에이전트는 1개의 가중치 세트를 이용하는데, 각 가중치의 값은 주어진 밴딧의 특정한 손잡이를 선택함으로써 반환되는 값의 예측값입니다. 우리는 정책 경사 방법을 사용하여 선택된 액션에 대한 값을 받은 보상으로 이동함으로써 에이전트를 업데이트합니다.

In [5]:
class agent():
    def __init__(self, lr, s_size,a_size):
        
        #네트워크의 피드-포워드 부분을 구축한다. 에이전트는 상태를 받아서 액션을 출력한다 
        self.state_in= tf.placeholder(shape=[1],dtype=tf.int32)
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
        output = slim.fully_connected(state_in_OH,a_size,\
            biases_initializer=None,activation_fn=tf.nn.sigmoid,weights_initializer=tf.ones_initializer())
        self.output = tf.reshape(output,[-1])
        self.chosen_action = tf.argmax(self.output,0)

        #학습 과정을 구축한다. 비용을 계산하기 위해 보상과 선택된 액션을 네트워크에 피드하고,
        #네트워크를 업데이트하는 데에 이를 이용한다.
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output,self.action_holder,[1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

# 에이전트 학습시키기

우리는 환경으로부터 상태를 받고 액션을 취하고 보상을 받음으로써 에이전트를 학습시킬 것입니다. 이 세가지를 이용함으로써, 주어진 상태에서 시간의 흐름에 따라 최고의 보상을 받을 수 있는 액션을 보다 자주 선택할 수 있도록 우리의 네트워크를 적절하게 업데이트시키는 방법에 대해 알 수 있습니다. 

In [6]:
tf.reset_default_graph() #텐서플로우 그래프를 리셋한다.

cBandit = contextual_bandit() #밴딧을 로드한다.
myAgent = agent(lr=0.001,s_size=cBandit.num_bandits,a_size=cBandit.num_actions) #에이전트를 로드한다.
weights = tf.trainable_variables()[0] #네트워크 내부를 들여다 보기 위해 평가할 가중치

total_episodes = 10000 #에이전트를 학습시킬 전체 에피소드 수 설정
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions]) #밴딧에 대한 점수표를 0으로 설정
e = 0.1 #랜덤한 액션을 취할 가능성 설정

init = tf.global_variables_initializer()

# 텐서플로우 그래프 론칭
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit() #환경으로부터 상태 가져오기
        
        #네트워크로부터 랜덤한 액션 또는 하나의 액션을 선택한다. 
        if np.random.rand(1) < e:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action,feed_dict={myAgent.state_in:[s]})
        
        reward = cBandit.pullArm(action) #주어진 밴딧 하에서 액션을 취한데 대한 보상을 얻는다. 
        
        #네트워크를 업데이트한다.
        feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update,weights], feed_dict=feed_dict)
        
        #총 보상을 업데이트한다.
        total_reward[s,action] += reward
        if i % 500 == 0:
            print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
        i+=1
for a in range(cBandit.num_bandits):
    print("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print("...and it was right!")
    else:
        print("...and it was wrong!")

Mean reward for each of the 3 bandits: [ 0.   -0.25  0.  ]
Mean reward for each of the 3 bandits: [ 31.75  38.5   40.5 ]
Mean reward for each of the 3 bandits: [ 74.    72.5   72.75]
Mean reward for each of the 3 bandits: [ 110.25  110.25  111.25]
Mean reward for each of the 3 bandits: [ 147.5   149.5   147.25]
Mean reward for each of the 3 bandits: [ 187.75  189.    182.  ]
Mean reward for each of the 3 bandits: [ 230.75  225.    216.5 ]
Mean reward for each of the 3 bandits: [ 271.5   261.5   250.75]
Mean reward for each of the 3 bandits: [ 311.5   298.75  285.  ]
Mean reward for each of the 3 bandits: [ 352.25  335.5   320.  ]
Mean reward for each of the 3 bandits: [ 390.5   377.5   356.75]
Mean reward for each of the 3 bandits: [ 434.5   415.5   392.25]
Mean reward for each of the 3 bandits: [ 466.    453.75  431.  ]
Mean reward for each of the 3 bandits: [ 507.25  492.5   467.  ]
Mean reward for each of the 3 bandits: [ 550.25  529.75  502.25]
Mean reward for each of the 3 bandits